# Neural Style Transfer - AI Essentials werkstuk - Charles D. White

##### the process of taking an image and reproducing it with a new artistic style using a pre-trained neural network

bronnen:
1. https: // arxiv .org/abs/1508.06576 - Leon A. Gatys, Alexander S. Ecker, Matthias Bethge


Style Transfer using the VGG19 Pre-Trained Model in PyTorch in order to calculate the content loss and the style loss

In [1]:
# uploading the style image
#Joaquim Falco Dali Portreto Picasso:
# https://auction.catawiki.com/kavels/28497261-falco-dali-contemplates-picasso-paintings
# Picasso satitirised his Sitters:
# https://www.apollo-magazine.com/picassos-portraits-satirised-his-sitters-but-also-art-itself/

In [2]:
# import libraries required
# basic data science and visualization libraries:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
# imports for the PyTorch libraries utilized:
import torch
import torch.optim as optim
from torchvision import models
from torchvision import transforms as tf
import torch.nn.functional as F

In [5]:
# Pre-trained model used for Style Transfer is VGG19 --> True => model comes initialized with pre-trained weights
# weights from pre-trained model applied need to remain frozen when performing neural style transfer on images --> False
# weights of model do not change; weights of target image will be trained content from content image and style from style image

In [12]:
vgg = models.vgg19(pretrained=True).features

for param in vgg.parameters():
    param.requires_grad_(False)

In [13]:
# downloads model weights and instantiates the VGG19 pretrained model with those weights
# then, we train target image for neural style transfer using a cuda device when possible
# this displays on the screen all of the layers of the VGG19 model stacked up sequentially
# every layer has a unique index starting at '0'
# there are convolutional layers, ReLU activations and pooling layers

In [14]:
device = torch.device("cpu")

if torch.cuda.is_available():
    device = torch.device("cuda")

vgg.to(device)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace)
  (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (17): ReLU(inplace)
  (18): MaxPool2d(kernel_size=2, stride=2, padding=0, 

In [17]:
# All pre-trained models in PyTorch expect input image pixels to be in [0,1] range normalized 
# using the mean and standard deviation, source: https://pytorch.org/docs/stable/torchvision/models.html
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)